In [91]:
# Imports
import numpy as np
import pandas as pd
from google.cloud import bigquery
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

import os

In [92]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

client = bigquery.Client(location="US", project="helio-staging")

Client creating using default project: helio-staging


In [93]:
rev_source = ["sephora","ulta"]

job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("revSource", "STRING", rev_source)
        ]
)
max_date_query = """
    SELECT DISTINCT execution_date
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    where execution_date > DATE_ADD(current_date(), INTERVAL -10 DAY)
    and normalized_url IS NOT NULL
    ORDER BY execution_date DESC
    LIMIT 2
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query
qj = query_job.to_dataframe()
max_date, prev_date = qj["execution_date"].astype('str')[0],qj["execution_date"].astype('str')[1]
 

In [111]:
prev_date ='2022-05-10'


In [132]:
max_date = '2022-05-11'

In [133]:
#max_date reviews
job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("rev_source", "STRING", rev_source),
            bigquery.ScalarQueryParameter("max_date", "STRING", max_date)
        ]
)
max_date_query = """
    SELECT source_name, normalized_url, min(review_date)
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    WHERE execution_date = @max_date
    AND source_name in UNNEST (@rev_source)
    Group by source_name, normalized_url
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

brands = query_job.to_dataframe()
brands = brands.loc[brands.astype(str).drop_duplicates().index]
brands = brands.rename(columns={"f0_": "earliest_review"})
brands = brands.sort_values(by = "earliest_review",ascending=False)
brands["brand_source"] = brands["source_name"] +  " " + brands["normalized_url"]

In [134]:
#prev_date reviews
job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("rev_source", "STRING", rev_source),
            bigquery.ScalarQueryParameter("prev_date", "STRING", prev_date)
        ]
)
max_date_query = """
    SELECT source_name, normalized_url, min(review_date), 
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    WHERE execution_date = @prev_date
    AND source_name in UNNEST (@rev_source)
    Group by source_name, normalized_url
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

prev_brands = query_job.to_dataframe()
prev_brands = prev_brands.loc[prev_brands.astype(str).drop_duplicates().index]
prev_brands = prev_brands.rename(columns={"f0_": "earliest_review"})
prev_brands = prev_brands.sort_values(by = "earliest_review",ascending=False)
prev_brands["brand_source"] = prev_brands["source_name"] +  " " + prev_brands["normalized_url"]

In [135]:
brands 


,source_name,normalized_url,earliest_review,brand_source
711,sephora,theoutset.com,2022-04-24 21:42:56,sephora theoutset.com
792,sephora,katemcleod.com,2022-04-14 00:10:46,sephora katemcleod.com
450,sephora,abbott.com,2022-04-04 17:41:14,sephora abbott.com
440,sephora,heyhanni.com,2022-03-29 10:04:45,sephora heyhanni.com
3,ulta,qhemetbiologics.com,2022-03-27 17:03:31.283,ulta qhemetbiologics.com
...,...,...,...,...
1079,sephora,clinique.com,2008-08-28 05:15:39,sephora clinique.com
871,ulta,philosophy.com,2008-07-27 07:00:00,ulta philosophy.com
356,ulta,covergirl.com,2007-07-20 01:14:16,ulta covergirl.com
1056,ulta,koparibeauty.com,2005-12-30 08:00:00,ulta koparibeauty.com


In [136]:
prev_brands

,source_name,normalized_url,earliest_review,brand_source
797,sephora,abbott.com,2022-04-04 17:41:14,sephora abbott.com
772,sephora,heyhanni.com,2022-03-29 10:04:45,sephora heyhanni.com
160,ulta,qhemetbiologics.com,2022-03-27 17:03:31.283,ulta qhemetbiologics.com
499,ulta,treslucebeauty.com,2022-03-07 14:20:34.865,ulta treslucebeauty.com
1093,sephora,cayskin.com,2022-03-01 09:12:45,sephora cayskin.com
...,...,...,...,...
701,sephora,clinique.com,2008-08-28 05:15:39,sephora clinique.com
1088,ulta,philosophy.com,2008-07-27 07:00:00,ulta philosophy.com
1166,ulta,covergirl.com,2007-07-20 01:14:16,ulta covergirl.com
852,ulta,koparibeauty.com,2005-12-30 08:00:00,ulta koparibeauty.com


In [129]:
brands[~brands["brand_source"].isin(prev_brands["brand_source"].tolist())]

,source_name,normalized_url,earliest_review,brand_source
480,sephora,theoutset.com,2022-04-24 21:42:56,sephora theoutset.com
765,sephora,katemcleod.com,2022-04-14 00:10:46,sephora katemcleod.com
629,ulta,pirettebeach.com,2020-07-25 21:53:15.25,ulta pirettebeach.com
58,ulta,arcticfox.com,2020-03-15 07:32:29.061,ulta arcticfox.com
668,ulta,flawlessbeauty.com,2018-03-06 08:00:00,ulta flawlessbeauty.com
751,sephora,shuuemura-usa.com,2015-09-12 16:34:46,sephora shuuemura-usa.com
577,ulta,catricecosmetics.com,2015-07-17 07:00:00,ulta catricecosmetics.com
349,ulta,ccleaner.com,2010-03-28 07:00:00,ulta ccleaner.com
882,ulta,ralphlaurenhome.com,2009-11-10 08:00:00,ulta ralphlaurenhome.com
88,ulta,giorgioarmanibeauty-usa.com,2009-11-05 08:00:00,ulta giorgioarmanibeauty-usa.com


In [137]:
new_brands = brands[~brands["brand_source"].isin(prev_brands["brand_source"].tolist())]
#drop earliest_review over a year ago
new_brands['earliest_review'] =  pd.to_datetime(new_brands['earliest_review'])

#yr_ago=datetime.now() - relativedelta(years=1)
qtr_ago = datetime.now() - relativedelta(months=3)
new_brands = new_brands[new_brands["earliest_review"]>qtr_ago]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [138]:
#filter by seen brands
#seen_brands = pd.read_csv('seen_brands.csv')

#new_brands = new_brands[~new_brands["brand_source"].isin(seen_brands["brand_source"].tolist())]

#seen_brands['brand_source'].append(new_brands["brand_source"])

#new_brands =new_brands.drop('brand_source',axis=1)

#seen_brands['brand_source'].to_csv('seen_brands.csv')

In [139]:
new_brands

,source_name,normalized_url,earliest_review,brand_source
711,sephora,theoutset.com,2022-04-24 21:42:56,sephora theoutset.com
792,sephora,katemcleod.com,2022-04-14 00:10:46,sephora katemcleod.com


In [87]:
mailing_list = ['tzeng-contractor@circleup.com',
                'twalker@circleup.com',
                'nmckinney@circleup.com',
                'sblumenthal@circleup.com', 
                'kweigand@circleup.com',
                'blee@circleup.com'
]

In [88]:
import smtplib
import ssl
from email.mime.text import MIMEText

In [89]:
#if new_brands not empty, create email report

filename = date.today().strftime("new_brands_%Y-%m-%d.csv")

if not new_brands.empty:
    print("New brands")
    #new_brands.to_csv(filename)
    new_brands_html = new_brands.to_html()
    
    msg = MIMEText(new_brands_html, "html")
    msg["From"] = "Helio Live Updates <tzeng-contractor@circleup.com>"
    msg["To"] = ','.join(mailing_list)
    msg["Subject"] = 'New to Sephora & Ulta ' + date.today().strftime("%Y-%m-%d")
    
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login("tzeng-contractor@circleup.com", "zualxbcbcuhgkvli")
    server.sendmail(
      "tzeng-contractor@circleup.com", 
      mailing_list, 
      msg.as_string() + "\n" + "\n" + "This is an automated message. Please reach out to Terry for any questions or changes.")
    server.quit()
